In [ ]:
!git clone https://github.com/omrimanor7/TrafficSignDetector.git


Cloning into 'TrafficSignDetector'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 108 (delta 12), reused 101 (delta 11), pack-reused 0
Receiving objects: 100% (108/108), 2.18 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
%cd TrafficSignDetector/Recognition/

/content/TrafficSignDetector/Recognition


In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
# !unzip "gdrive/MyDrive/EE Final Project/GTSRB/GTSRB_Final_Test_Images.zip" -d "gdrive/MyDrive/EE Final Project/GTSRB"
# !unzip "gdrive/MyDrive/EE Final Project/GTSRB/GTSRB-Training_fixed.zip" -d "gdrive/MyDrive/EE Final Project/GTSRB"
# !unzip "/content/gdrive/MyDrive/EE Final Project/GTSRB/GTSRB_Final_Test_GT.zip" -d "/content/gdrive/MyDrive/EE Final Project/GTSRB"

In [ ]:
!python "train_model.py" -a="resnet50" -v="gdrive/MyDrive/EE Final Project/GTSRB/GTSRB/Final_Test/Images" -t="gdrive/MyDrive/EE Final Project/GTSRB/GTSRB/Training" -l="gdrive/MyDrive/EE Final Project/GTSRB/GTSRB/Final_Test/GT-final_test.csv" 

2021-04-17 12:14:10.493660: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Traceback (most recent call last):
  File "train_model.py", line 235, in <module>
    raise IOError("Cannot read directory {}".format(args.train_path))
OSError: Cannot read directory gdrive/MyDrive/EE Final Project/GTSRB/GTSRB/Training


#Imports

In [ ]:
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger
from preprocessing import get_dataset
from train_model import *
from tensorflow.keras.utils import to_categorical
from tensorflow import convert_to_tensor

#Train Model

In [ ]:
architecture="resnet50"
train_path="/content/gdrive/MyDrive/EE Final Project/datasets/GTSRB/GTSRB/Training"
test_path="/content/gdrive/MyDrive/EE Final Project/datasets/GTSRB/GTSRB/Final_Test/Images"
test_labels_path="/content/gdrive/MyDrive/EE Final Project/datasets/GTSRB/GTSRB/Final_Test/GT-final_test.csv"
img_size=64
result_folder="/content/gdrive/MyDrive/EE Final Project/Models/Classifier"
grayscale=False

In [ ]:
xtrain, ytrain, xtest, ytest = get_dataset(train_path, test_path, test_labels_path, (img_size, img_size), grayscale)

number of classes: 43
shape of train images: (26640, 64, 64, 3)
shape of labels: (26640,)
number of test images: 12630
shape of test images: (12630, 64, 64, 3)
shape of test labels: (12630,)


In [ ]:
def build_resnet50(num_classes, img_size):
    from tensorflow.keras.applications.resnet import ResNet50
    from tensorflow.keras import Model
    from tensorflow.keras.layers import Dense, Flatten
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=img_size)
    x = Flatten(input_shape=resnet.output.shape)(resnet.output)
    x = Dense(1024, activation='sigmoid')(x)
    predictions = Dense(num_classes, activation='softmax', name='pred')(x)
    model = Model(inputs=[resnet.input], outputs=[predictions])
    return model
  
def train_model(model, xtrain, ytrain, xtest, ytest, lr=0.001, batch_size=32, epochs=10, result_folder=""):
    """
    Trains a CNN for a given dataset
    :param model: initialized model
    :param xtrain: training images
    :param ytrain: labels for training images numbered from 0 to n
    :param xtest: test images
    :param ytest: labels for test images numbered from 0 to n
    :param lr: initial learning rate for SGD optimizer
    :param batch_size: batch size
    :param epochs: number of epochs to train
    :param result_folder: Save trained model to this directory
    :return: None
    """
    from tensorflow.keras.optimizers import SGD
    from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
    sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    checkpoint = ModelCheckpoint(os.path.join(result_folder, "model.h5"), save_best_only=True)
    csv_logger = CSVLogger(os.path.join(result_folder, "training.log"), separator=",", append=True)
    history = model.fit(xtrain, ytrain,
                        batch_size=batch_size,
                        validation_data=(xtest, ytest),
                        epochs=epochs,
                        callbacks=[LearningRateScheduler(lr_schedule), checkpoint, csv_logger])

    plot(history, result_folder)

In [ ]:
num_classes = np.unique(ytrain).size
ytrain = to_categorical(ytrain, num_classes=num_classes)
ytest = to_categorical(ytest, num_classes=num_classes)

model_input_shape = xtrain[0].shape
model = None
# train model
if architecture == 'vgg19':
    model = build_vgg19(num_classes, model_input_shape)
elif architecture == 'lenet-5':
    model = build_lenet5(num_classes, model_input_shape)
elif architecture == 'alex':
    model = build_alexnet(num_classes, model_input_shape)
elif architecture == 'resnet50':
    model = build_resnet50(num_classes, model_input_shape)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
if model != None:
    train_model(model, convert_to_tensor(xtrain), ytrain, convert_to_tensor(xtest), ytest, lr=0.001, batch_size=32, epochs=4, result_folder=result_folder)
else:
    "model architecture not implemented yet"

#Testing the Model